In [1]:
! pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 3.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 285.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=a04046cf59e4b0b39ca6f241720720be3dfbffae0cd82fdb5c3cd9f8df598560
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [35]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df=pd.read_excel("train.xlsx")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,reason,label
0,this is an amazing app for online classes!but,good app for conducting online classes,1
1,very practical and easy to use,app is user-friendly,1
2,this app is very good for video conferencing.,good for video conferencing,1
3,i can not download this zoom app,unable to download zoom app,1
4,i am not able to download this app,want to download the app,1
...,...,...,...
2056,i am not getting jio tv properly.,unable to use with jio tv,1
2057,(learn from the netflix interface),netflix ui is better,1
2058,"hello,\ndisney+ must also be installed on chro...",unable to use disney on chromecast,1
2059,it is a shame that disney+ does not work on tv...,unable to use disney plus on tv,1


In [19]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [36]:
from sentence_transformers import  InputExample,losses
from torch.utils.data import DataLoader
train_examples = []
text=list(df['text'].astype('str'))
reason=list(df['reason'].astype('str'))
labl=list(df['label'].astype('float'))
for i in range(len(df)):
  train_examples.append(InputExample(texts=[text[i],reason[i]], label=labl[i]))
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(model)

In [32]:
from sentence_transformers import evaluation
labl_eval=list(df['label'])
evaluator= evaluation.BinaryClassificationEvaluator(text[:500],reason[:500],labl_eval[:500],write_csv=True)

In [33]:
reason[type(reason)=='float']
type(reason[0])

str

In [37]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/258 [00:00<?, ?it/s]

Iteration:   0%|          | 0/258 [00:00<?, ?it/s]

In [38]:
test_df=pd.read_excel("evaluation.xlsx")
text1=list(test_df['text'])
text2=list(test_df['reason'])
text1_en=model.encode(text1)
text2_en=model.encode(text2)
test_labl=list(test_df['label'])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(test_df.shape[0]):
  test_df.loc[i,'output']=cosine_similarity([text1_en[i]],[text2_en[i]])[0][0]

test_df.to_excel('/content/drive/MyDrive/output_del_elem2.xlsx')